<a href="https://colab.research.google.com/github/ptleskin/exerciseData/blob/master/Implementation_of_%22Simple_and_accurate_analytical_calculation_of_shortest_path_lengths%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install dependencies

In [0]:
!pip install numpy networkx

Import libraries

In [0]:
import collections
import networkx as nx
import numpy as np
import random
from itertools import product
from math import exp


Equation (4) for estimating D_n for a z-regular random network:

In [0]:
def calculate_regular(G):
    N = G.number_of_nodes()
    p_k = getDD(G)
    
    z = sum([k*p for k,p in p_k.items()])
    
    D_n = [0.0]
    iters = 32
    
    for n in range(1,iters):
        
        d_n = D_exp(n-1, N, z) - D_exp(n, N, z)
        
        D_n.append(d_n)
    
    sumd = sum(D_n)
    expD = sum([n*d/sumd for n,d in enumerate(D_n)])
    return expD

def D_exp(n,N,z):
    return exp(-(z*((z-1)**n) -2)/(N*(z-2)))

Implementation of calculations in Section IV: p_k-theory

In [0]:
def calculate_random(G):
    N = G.number_of_nodes()
    p_k = getDD(G)
    pk = list(p_k.keys())
    
    z = sum([k*p for k,p in p_k.items()])
    
    iters = 32
    rho, q = [], []
    q_bar = {}
    
    for n in range(iters):
        
        rho.append(dict([((k,_k), 0.0) for k,_k in product(pk,pk)]))
        q.append(dict([((k,_k), 0.0) for k,_k in product(pk,pk)]))
        
        if n==0:
            
            for k in pk:
                rho_0 = 1/(N*p_k[k])
                rho[0][(k,k)] = rho_0
                q[0][(k,k)] = rho_0
            
            q_bar = dict([(_k,0.0) for _k in pk])
            for (k,_k), _q in q[n].items():
                q_bar[_k] += k*p_k[k]*_q
            
            for _k in q_bar.keys():
                q_bar[_k] /= z
        else:
            
            j = n-1
            
            rho[n] = dict([((k,_k),1-(1-rho[0][(k,_k)])*((1-q_bar[_k])**k)) 
                        for k,_k in q[j].keys()])
            q[n] = dict([((k,_k),1-(1-rho[0][(k,_k)])*((1-q_bar[_k])**(k-1))) 
                        for k,_k in q[j].keys()])
            
            q_bar = dict([(_k,0.0) for _k in pk])
            for (k,_k), _q in q[n].items():
                q_bar[_k] += k*p_k[k]*_q
            
            for _k in q_bar.keys():
                q_bar[_k] /= z
            
            
    
    D_n = [0.0]
    for n in range(1,iters):
        D_nkk = dict([ (
                    (k,_k),
                    (_d-rho[n-1][(k,_k)])/rho[-1][(k,_k)] ) 
                    for (k,_k), _d in rho[n].items() ])
        d_n = sum([p_k[k]*p_k[_k]*_d for (k,_k), _d in D_nkk.items()] )
        D_n.append(d_n)
    
    sumd = sum(D_n)
    expD = sum([n*d/sumd for n,d in enumerate(D_n)])
    
    return expD

Implementation of calculations in Section V: P(k,k')-theory

In [0]:
def calculate_ByJDD(G):
    N = G.number_of_nodes()
    p_k = getDD(G)
    
    Pkk = getJDD(G)
    pk = list(set([x for x,_ in Pkk.keys()]+[x for _,x in Pkk.keys()]))
    
    iters = 32
    rho, q = [], []
    q_bar = {}
    
    for n in range(iters):
        
        rho.append(dict([((k,_k), 0.0) for k,_k in Pkk.keys()]))
        q.append(dict([((k,_k), 0.0) for k,_k in Pkk.keys()]))
        
        if n==0:
            
            for k in pk:
                rho_0 = 1/(N*p_k[k])
                rho[0][(k,k)] = rho_0
                q[0][(k,k)] = rho_0
                
        else:
            
            j = n-1
            
            q_bar = dict([((k,_k), [0.0, 0.0]) for k,_k in Pkk.keys()])
            
            for (k,_k), _p in Pkk.items():
                for k2 in pk:
                    try:
                        q_bar[(k,_k)][1] += Pkk[(k,k2)]
                        q_bar[(k,_k)][0] += Pkk[(k,k2)]*q[j][(k2,_k)]
                    except KeyError:
                        pass
                    
            for k,_k in Pkk.keys():
                if q_bar[(k,_k)][1]>0:
                    q_bar[(k,_k)][0] /= q_bar[(k,_k)][1]
            
            rho[n] = dict([((k,_k),1-(1-rho[0][(k,_k)])*((1-q_bar[(k,_k)][0])**k)) 
                        for k,_k in Pkk.keys()])
            q[n] = dict([((k,_k),1-(1-rho[0][(k,_k)])*((1-q_bar[(k,_k)][0])**(k-1))) 
                        for k,_k in Pkk.keys()])
            
            
            if False:
                sum_rho = sum(rho[n].values())
                sum_q = sum(q[n].values())
                print("n {}".format(n))
                print("Sum rho {}".format(sum_rho))
                print("Sum q {}".format(sum_q))
    
    D_n = [0.0]
    
    for n in range(1,iters):
        D_nkk = dict([ (
                    (k,_k),
                    (_d-rho[n-1][(k,_k)])/rho[-1][(k,_k)] ) 
                    for (k,_k), _d in rho[n].items() ])
        
        d_n = sum([p_k[k]*p_k[_k]*_d for (k,_k), _d in D_nkk.items()] )
        D_n.append(d_n)
        
    sumd = sum(D_n)
    expD = sum([n*d/sumd for n,d in enumerate(D_n)])
    
    return expD



Define some useful tools:

In [0]:
# calcullate estimate for input graph G
def analyzeGraph(G):    
    N = G.number_of_nodes()
    print("Number of nodes: {}".format(N))
    E = G.number_of_edges()
    print("Average degree: {}".format(2.0*E/N))
    d_bar = nx.average_shortest_path_length(G)
    
    expD = calculate_random(G)
    print("Average shortest path length: {:.4f}".format(d_bar))
    print("Expected D with p_k\t{:.4f}".format(expD))
    print("Relative error\t{:.4f}\n".format(relative_error(d_bar, expD)))
    
    
    expD = calculate_ByJDD(G)
    print("Expected D with P(k,k')\t{:.4f}".format(expD))
    print("Relative error\t{:.4f}\n".format(relative_error(d_bar, expD)))
    
    D_rrg = calculate_regular(G)
    print("Regular degree estimate\t{:.4f}".format(D_rrg))
    print("Relative error\t{:.4f}".format(relative_error(d_bar, D_rrg)))
    
# calculate joint degree-degree distribution of input network G
def getJDD(G):
    M = nx.degree_mixing_matrix(G.to_directed(), normalized=True)
    Pkk = dict([((k,k_),x) for k,row in enumerate(M) for k_,x in enumerate(row) if x>0])
    return Pkk

# calculate degree distribution of input network G
def getDD(G):
    degree_sequence = sorted([d for n, d in G.degree()], reverse=True)  # degree sequence
    degrees = collections.Counter(degree_sequence)
    N = sum(degrees.values())
    return dict([(x,y/N) for x,y in degrees.items()])

# calculate relative error of v_approx to v
def relative_error(v,v_approx):
    return abs((v-v_approx)/v)

First run a test with a random 4-regular graph with N=500

In [0]:
G = nx.random_regular_graph(n=500,d=4)

analyzeGraph(G)

Number of nodes: 500
Average degree: 4.0
Average shortest path length: 5.0013
Expected D with p_k	5.0214
Relative error	0.0040

Expected D with P(k,k')	5.0214
Relative error	0.0040

Regular degree estimate	5.0223
Relative error	0.0042


In [0]:
G = nx.random_regular_graph(n=500,d=10)
analyzeGraph(G)

Number of nodes: 500
Average degree: 10.0
Average shortest path length: 2.9472
Expected D with p_k	2.9605
Relative error	0.0045

Expected D with P(k,k')	2.9605
Relative error	0.0045

Regular degree estimate	2.9610
Relative error	0.0047


A larger random 4-regular graph with N=10000

In [0]:
G = nx.random_regular_graph(n=10000,d=4)

analyzeGraph(G)

Number of nodes: 10000
Average degree: 4.0
Average shortest path length: 7.7255
Expected D with p_k	7.7288
Relative error	0.0004

Expected D with P(k,k')	7.7288
Relative error	0.0004

Regular degree estimate	7.7288
Relative error	0.0004


A larger random 10-regular graph with N=10000

In [0]:
G = nx.random_regular_graph(n=10000,d=10)

analyzeGraph(G)

Number of nodes: 10000
Average degree: 10.0
Average shortest path length: 4.3421
Expected D with p_k	4.3431
Relative error	0.0002

Expected D with P(k,k')	4.3431
Relative error	0.0002

Regular degree estimate	4.3431
Relative error	0.0002


Then a Erdos-Renyi graph with N=500, p=0.02

In [0]:
G = nx.erdos_renyi_graph(500,0.02)
analyzeGraph(G)

Number of nodes: 500
Average degree: 9.932
Average shortest path length: 2.9425
Expected D with p_k	2.9592
Relative error	0.0056

Expected D with P(k,k')	2.9598
Relative error	0.0059

Regular degree estimate	2.9696
Relative error	0.0092


In [0]:
G = nx.erdos_renyi_graph(10000,0.001)
analyzeGraph(G)

Number of nodes: 10000
Average degree: 9.9766
Average shortest path length: 4.2633
Expected D with p_k	4.2623
Relative error	0.0002

Expected D with P(k,k')	4.2627
Relative error	0.0002

Regular degree estimate	4.3474
Relative error	0.0197


Then C.elegans Neural network

source: https://www.complex-networks.net/datasets.html#c_elegans

In [0]:
!wget https://raw.githubusercontent.com/ptleskin/exerciseData/master/data/c_elegans_undir.net?raw=true
!mv c_elegans_undir.net\?raw\=true c_elegans_undir.net

G=nx.read_weighted_edgelist("c_elegans_undir.net", nodetype=int)
analyzeGraph(G)

--2019-02-24 11:51:58--  https://raw.githubusercontent.com/ptleskin/exerciseData/master/data/c_elegans_undir.net?raw=true
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16186 (16K) [text/plain]
Saving to: ‘c_elegans_undir.net?raw=true’

c_elegans_undir.net 100%[===================>]  15.81K  --.-KB/s    in 0.01s   

2019-02-24 11:51:58 (1.61 MB/s) - ‘c_elegans_undir.net?raw=true’ saved [16186/16186]

Number of nodes: 279
Average degree: 16.39426523297491
Average shortest path length: 2.4356
Expected D with p_k	2.3318
Relative error	0.0426

Expected D with P(k,k')	2.2932
Relative error	0.0585

Regular degree estimate	2.3245
Relative error	0.0456


500 US Airports network

source: https://www.complex-networks.net/datasets.html#airports

In [0]:
!wget https://raw.githubusercontent.com/ptleskin/exerciseData/master/data/US_airports.net?raw=true
!mv US_airports.net\?raw\=true US_airports.net

G=nx.read_weighted_edgelist("US_airports.net", nodetype=int)
analyzeGraph(G)

--2019-02-24 11:53:30--  https://raw.githubusercontent.com/ptleskin/exerciseData/master/data/US_airports.net?raw=true
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39832 (39K) [text/plain]
Saving to: ‘US_airports.net?raw=true’

US_airports.net?raw 100%[===================>]  38.90K  --.-KB/s    in 0.01s   

2019-02-24 11:53:30 (3.85 MB/s) - ‘US_airports.net?raw=true’ saved [39832/39832]

Number of nodes: 500
Average degree: 11.92
Average shortest path length: 2.9910
Expected D with p_k	2.8022
Relative error	0.0631

Expected D with P(k,k')	2.8559
Relative error	0.0452

Regular degree estimate	2.7729
Relative error	0.0729
